In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import warnings
warnings.filterwarnings('ignore')

print('All the libraries imported!')

All the libraries imported!


In [54]:
df_raw=pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
df_raw.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200618,AK,708.0,80477.0,NaN,18.0,NaN,NaN,NaN,0.0,NaN,449.0,A,6/18/2020 00:00,2020-06-18T00:00:00Z,06/17 20:00,12.0,NaN,2020-06-18T00:00:00Z,81185.0,NaN,NaN,NaN,2,12,3464,81185,81185,3476,81185,0,0,db312ffaaa17d8bf8435e7fa3f58db1ece2567ff,0,0,0,0,0,NaN
1,20200618,AL,28206.0,294328.0,NaN,663.0,2373.0,NaN,695.0,NaN,405.0,15974.0,B,6/18/2020 11:00,2020-06-18T11:00:00Z,06/18 07:00,810.0,2373.0,2020-06-18T11:00:00Z,NaN,NaN,NaN,27796.0,1,894,10917,322534,322534,11811,322534,20,21,51542bad98c97fde34e7dc5472ef9416c6822a69,0,0,0,0,0,NaN
2,20200618,AR,13928.0,214506.0,NaN,226.0,1074.0,NaN,NaN,53.0,173.0,9376.0,B,6/18/2020 15:10,2020-06-18T15:10:00Z,06/18 11:10,208.0,1074.0,2020-06-18T15:10:00Z,NaN,NaN,NaN,13928.0,5,322,7413,228434,228434,7735,228434,11,22,18689a56d425e2051c51c144e355569ce4786172,0,0,0,0,0,NaN
3,20200618,AS,0.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,6/1/2020 00:00,2020-06-01T00:00:00Z,05/31 20:00,0.0,NaN,2020-06-01T00:00:00Z,NaN,NaN,NaN,NaN,60,0,0,174,174,0,174,0,0,b514616f295b79fb24ef69691cc1f69261b6182a,0,0,0,0,0,NaN
4,20200618,AZ,43443.0,336289.0,NaN,1667.0,3956.0,540.0,NaN,341.0,NaN,6863.0,A+,6/18/2020 00:00,2020-06-18T00:00:00Z,06/17 20:00,1271.0,3956.0,2020-06-18T00:00:00Z,379374.0,NaN,NaN,43085.0,4,2519,11020,379732,379732,13539,379732,32,95,daaa2730525a7a7acc7d1498d3145f89c28d1ef1,0,0,0,0,0,NaN


In [75]:
# Columns to keep
df_2=df_raw[['date','state','positive']]

In [78]:
# Converting date to datetime
df_2['DateTime'] = pd.to_datetime(df_2['date'].astype(str), format='%Y%m%d');
df_2.head()

,date,state,positive,DateTime
0,20200618,AK,708.0,2020-06-18
1,20200618,AL,28206.0,2020-06-18
2,20200618,AR,13928.0,2020-06-18
3,20200618,AS,0.0,2020-06-18
4,20200618,AZ,43443.0,2020-06-18


In [79]:
scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], \
       [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds
data_slider = []

In [80]:
#### I populate the data object
vect=df_2.date.unique()
for i in range(len(vect)-1):
    i=i+1
    # I select the year (and remove DC for now)
    df_sected_1 = df_2[(df_2['state']!= 'DC' ) &  (df_2['date']== vect[i] )]
    #print(df_sected_1.shape,df_sected_1.head())
    df_sected = df_2[(df_2['state']!= 'DC' ) &  (df_2['date']== vect[i-1] )]
    #print(df_sected.shape,df_sected.head())
    #print(df_sected_1['positive'])
    #print(df_sected['positive'])
    try:
      df_sected['positive_diff']=df_sected['positive'].values-df_sected_1['positive'].values
    except:
      df_sected['positive_diff']=df_sected['positive'].sub(pd.Series(df_sected_1['positive'].values, index=df_sected_1['state'].values),axis='index')
    #print(df_sected.head())
      

    for col in df_sected.columns:  # I transform the columns into string type so I can:
        df_sected[col] = df_sected[col].astype(str)

    ### I create the text for mouse-hover for each state, for the current year    
    #df_sected['text'] = df_sected['state'] + 'Pop: ' /span> df_sected['psotive']'Positive Cases: '+df_sected_crime['positive']

    ### create the dictionary with the data for the current year
    data_one_date = dict(
        type='choropleth',
        locations = df_sected['state'],
        z=df_sected['positive_diff'].astype(float),
        locationmode='USA-states',
        colorscale = scl,
        #text = df_sected_crime['text'],
        )

    data_slider.append(data_one_date)  # I add the dictionary to the list of dictionaries for the slider


In [81]:
len(data_slider)
#data_slider[400]
timeVector=[]
timeVector= np.flip(pd.date_range(df_2['DateTime'].iloc[-2],df_2['DateTime'].iloc[0],freq='1D'))

In [82]:
steps = []

for i in range(len(timeVector)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label=format( str(timeVector.month[i])+'-'+str(timeVector.day[i])  ) ) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)] 

In [70]:

layout = dict(
    title = 'Daily Positive Covid-19 Cases in the US',
    geo=dict(scope='usa',
             projection={'type': 'albers usa'}),
              sliders=sliders)

fig = dict(data=data_slider, layout=layout) 

plotly.offline.iplot(fig)

offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_covid_slider" ,image_width=2000, image_height=1000, filename='map_us_covid_slider.html', validate=True)


'map_us_covid_slider.html'